# Final Notebook:
This notebook contains a the final data extraction, cleaning, feature engineering and modele training processes

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
pd.set_option('display.max_columns', None)

# Read dataset

In [24]:
df = pd.read_csv('pca.csv')

In [25]:
df

,0,1,cluster
0,0.120225,-0.509557,0
1,0.396926,0.669551,0
2,-0.482344,-1.025046,0
3,-2.866458,0.282928,0
4,-1.869160,1.320057,0
...,...,...,...
7769,1.648740,-0.971848,-1
7770,-1.483258,-0.219563,0
7771,0.476767,-1.378634,0
7772,0.212460,0.581278,0


In [27]:
df['0']

0       0.120225
1       0.396926
2      -0.482344
3      -2.866458
4      -1.869160
          ...   
7769    1.648740
7770   -1.483258
7771    0.476767
7772    0.212460
7773   -2.167984
Name: 0, Length: 7774, dtype: float64

In [21]:
data = pd.read_csv('test.csv')

In [22]:
data

,0,1,cluster
0,0.120225,-0.509557,0
1,0.396926,0.669551,0
2,-0.482344,-1.025046,0
3,-2.866458,0.282928,0
4,-1.869160,1.320057,0
...,...,...,...
7769,1.648740,-0.971848,0
7770,-1.483258,-0.219563,0
7771,0.476767,-1.378634,0
7772,0.212460,0.581278,0


In [23]:
data['cluster'].value_counts()

cluster
 0    7498
-1     276
Name: count, dtype: int64

In [26]:
raw_data = pd.read_csv('raw_data.csv')

In [31]:
raw_data['cluster'].value_counts()

cluster
 0    7446
-1     328
Name: count, dtype: int64

In [27]:
raw_data

,raw_log,user_id,timestamp,action,amount,currency,location,device,amount_in_dollars,date,hour,day,month,year,txn_time_diff_hours,is_same_location,is_atm,transaction_risk,device_type,is_same_device,hour_risk,avg_prev_amount,amount_difference_from_avg,cluster
0,2025-06-01 12:03:31 - user=user1000 - action=c...,user1000,2025-06-01 12:03:31,cashout,2981.21,$,london,samsung galaxy s10,2981.2100,2025-06-01,12,1,6,2025,-1.000000,False,True,high,smart,False,Low,2981.210000,0.000000,0
1,01/06/2025 19:19:50 ::: user1000 *** debit :::...,user1000,2025-06-01 19:19:50,debit,1267.67,€,manchester,xiaomi mi 11,2028.2720,2025-06-01,19,1,6,2025,7.271944,False,False,moderate,smart,False,Low,2981.210000,-952.938000,0
2,2025-06-02 19:52:44 | user: user1000 | txn: re...,user1000,2025-06-02 19:52:44,refund,3610.68,$,cardiff,huawei p30,3610.6800,2025-06-02,19,2,6,2025,24.548333,False,False,high,smart,False,Low,2504.741000,1105.939000,0
3,2025-06-03 10:11:53 - user=user1000 - action=c...,user1000,2025-06-03 10:11:53,cashout,4659.06,£,birmingham,nokia 3310,6196.5498,2025-06-03,10,3,6,2025,14.319167,False,True,high,non-smart,False,Low,2873.387333,3323.162467,0
4,2025-06-03 21:23:30 | user: user1000 | txn: ca...,user1000,2025-06-03 21:23:30,cashout,4063.97,£,liverpool,none,5405.0801,2025-06-03,21,3,6,2025,11.193611,False,False,high,non-smart,False,Low,3704.177950,1700.902150,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7769,user1099 2025-07-24 14:45:45 refund 1420.36 li...,user1099,2025-07-24 14:45:45,refund,1420.36,none,liverpool,none,1420.3600,2025-07-24,14,24,7,2025,148.348333,True,False,high,non-smart,False,Low,2610.405450,-1190.045450,-1
7770,2025-07-25 10:04:10 | user: user1099 | txn: de...,user1099,2025-07-25 10:04:10,debit,2847.84,€,none,pixel 6,4556.5440,2025-07-25,10,25,7,2025,19.306944,False,False,moderate,smart,True,Low,2578.242059,1978.301941,0
7771,2025-07-25 18:10:43::user1099::refund::2876.11...,user1099,2025-07-25 18:10:43,refund,2876.11,none,birmingham,iphone 13,2876.1100,2025-07-25,18,25,7,2025,8.109167,False,False,high,smart,False,Low,2630.302637,245.807363,0
7772,2025-07-27 11:55:14 - user=user1099 - action=d...,user1099,2025-07-27 11:55:14,debit,1299.55,€,none,iphone 13,2079.2800,2025-07-27,11,27,7,2025,41.741944,False,True,moderate,smart,False,Low,2636.605390,-557.325390,0


In [21]:
data

,currency,transaction_risk,action_cashout,action_debit,action_deposit,action_purchase,action_refund,action_top-up,action_transfer,action_withdrawal,location_birmingham,location_cardiff,location_glasgow,location_leeds,location_liverpool,location_london,location_manchester,location_none,device_huawei p30,device_iphone 13,device_nokia 3310,device_none,device_pixel 6,device_samsung galaxy s10,device_xiaomi mi 11,is_atm_True,is_same_device_True,hour_risk_Low,device_type_smart,is_same_location_True,amount_in_dollars_scaled,hour_scaled,day_scaled,txn_time_diff_hours_scaled,avg_prev_amount_scaled,amount_difference_from_avg_scaled,cluster
0,1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.025331,0.080233,-1.710310,-0.797160,-0.094861,-0.000812,0
1,2,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.526651,1.088814,-1.710310,-0.394121,-0.094861,-0.491194,0
2,1,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.305819,1.088814,-1.594272,0.447646,-1.065344,0.568304,0
3,3,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.666187,-0.207932,-1.478233,-0.050755,-0.314477,1.709288,0
4,3,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.249813,1.376979,-1.478233,-0.203043,1.377696,0.874472,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7769,0,3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.846460,0.368399,0.958581,6.479625,-0.850124,-0.613210,-1
7770,2,2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.803417,-0.207932,1.074620,0.192267,-0.915635,1.017222,0
7771,0,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,-0.080622,0.944731,1.074620,-0.353328,-0.809597,0.125680,0
7772,2,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,-0.499817,-0.063849,1.306697,1.285381,-0.796759,-0.287612,0


In [22]:
data.shape

(7774, 37)

In [23]:
data.isnull().sum()

currency                             0
transaction_risk                     0
action_cashout                       0
action_debit                         0
action_deposit                       0
action_purchase                      0
action_refund                        0
action_top-up                        0
action_transfer                      0
action_withdrawal                    0
location_birmingham                  0
location_cardiff                     0
location_glasgow                     0
location_leeds                       0
location_liverpool                   0
location_london                      0
location_manchester                  0
location_none                        0
device_huawei p30                    0
device_iphone 13                     0
device_nokia 3310                    0
device_none                          0
device_pixel 6                       0
device_samsung galaxy s10            0
device_xiaomi mi 11                  0
is_atm_True              

In [24]:

features = [
    'action', 'currency',
    'location', 'device','amount_in_dollars', 'hour', 'day',
        'txn_time_diff_hours', 'is_same_location', 'is_atm',
    'transaction_risk', 'device_type', 'is_same_device', 'hour_risk', 
    'avg_prev_amount', 'amount_difference_from_avg'
]

In [25]:
df = data[features]

KeyError: "['action', 'location', 'device', 'amount_in_dollars', 'hour', 'day', 'txn_time_diff_hours', 'is_same_location', 'is_atm', 'device_type', 'is_same_device', 'hour_risk', 'avg_prev_amount', 'amount_difference_from_avg'] not in index"

In [ ]:
df

,action,currency,location,device,amount_in_dollars,hour,day,txn_time_diff_hours,is_same_location,is_atm,transaction_risk,device_type,is_same_device,hour_risk,avg_prev_amount,amount_difference_from_avg
0,cashout,$,london,samsung galaxy s10,2981.2100,12,1,-1.000000,False,True,high,smart,False,Low,2981.210000,0.000000
1,debit,€,manchester,xiaomi mi 11,2028.2720,19,1,7.271944,False,False,moderate,smart,False,Low,2981.210000,-952.938000
2,refund,$,cardiff,huawei p30,3610.6800,19,2,24.548333,False,False,high,smart,False,Low,2504.741000,1105.939000
3,cashout,£,birmingham,nokia 3310,6196.5498,10,3,14.319167,False,True,high,non-smart,False,Low,2873.387333,3323.162467
4,cashout,£,liverpool,none,5405.0801,21,3,11.193611,False,False,high,non-smart,False,Low,3704.177950,1700.902150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7769,refund,none,liverpool,none,1420.3600,14,24,148.348333,True,False,high,non-smart,False,Low,2610.405450,-1190.045450
7770,debit,€,none,pixel 6,4556.5440,10,25,19.306944,False,False,moderate,smart,True,Low,2578.242059,1978.301941
7771,refund,none,birmingham,iphone 13,2876.1100,18,25,8.109167,False,False,high,smart,False,Low,2630.302637,245.807363
7772,debit,€,none,iphone 13,2079.2800,11,27,41.741944,False,True,moderate,smart,False,Low,2636.605390,-557.325390
